In [22]:
from lxml import etree
import sqlite3

In [23]:
xml_db = 'wordnet/data/xml_synsets_new.db'
entry_db = 'wordnet/data/xml_entries.db'

In [20]:
tree = etree.parse("estwn-et-2.0.0.beta.xml")

In [21]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except sqlite3.Error as e:
        print("Connection error: [%s]" % e)

    return None

def create_table(conn, create_table_sql ):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except sqlite3.Error as e:
        print("Connection error while creating table: [%s]" % e)

def synsetTable(databaseLoc):

    sql_create_synset_table = ''' CREATE TABLE IF NOT EXISTS xml_synsets(
                                        
                                        id INT NOT NULL,
                                        estwn_id TEXT NOT NULL,
                                        sourceSense TEXT NOT NULL,
                                        pos TEXT NOT NULL,
                                        sense INT NOT NULL,
                                        writtenForm TEXT NOT NULL
                                                    ); '''
    conn = create_connection(databaseLoc)
    if conn is not None:
        create_table(conn,sql_create_synset_table)
    else:
        print("Error! cannot create db conn.")  
        

In [27]:
def upload_data(entry_xml_db,sset_xml_db):
    '''
    fetches pos sense and writtenForm from xml db, then with estwn id AND vertex id uploads it to xml_synsets.db
    '''
    synsetTable(sset_xml_db)
    conn = create_connection(sset_xml_db)
    cursor = conn.cursor()

    with conn:
        conn2 = create_connection(entry_xml_db)
        cursor2 = conn2.cursor()
        with conn2:
            i=0
            for estId in estwnId_list:
                cursor2.execute("SELECT pos,sense,writtenForm FROM xml_LexicalEntry WHERE estwn_id = ?", (estId,))
                row = cursor2.fetchall()
                    
                if len(row) > 1:
                    if sset_word[i] == 'None':
                        try:
                            var = row[0][2]
                            sourceSense = var.replace("_"," ")
                        except:
                            sourceSense = row[0][2]
                    else:
                        sourceSense = sset_word[i]
                    for word in row:
                        cursor.execute("INSERT INTO xml_synsets(id, estwn_id, sourceSense, pos, sense, writtenForm) VALUES (?,?,?,?,?,?)"\
                                                                                  ,(i,estwnId_list[i],sourceSense, word[0],word[1],word[2]))
                    conn.commit()
                else:
                    if sset_word[i] == 'None':
                        try:
                            var = row[0][2]
                            sourceSense = var.replace("_"," ")
                        except:
                            sourceSense = row[0][2]
                    else:
                        sourceSense = sset_word[i]
                    cursor.execute("INSERT INTO xml_synsets(id, estwn_id, sourceSense, pos, sense, writtenForm) VALUES (?,?,?,?,?,?)"\
                                                                                  ,(i,estwnId_list[i],sourceSense,row[0][0],row[0][1],row[0][2]))
                    conn.commit()
                i+=1

In [11]:

def fetch_estwnId(db_file):
    '''
    Fetches from xml database estwn id and sourceSense. Appends to list.
    '''

    synsets = tree.xpath('/LexicalResource/Lexicon/Synset')
    j=0
    for sset in synsets:
        estwnId_list.append(sset.attrib['id'])
        try:
            var   = sset[0].attrib['sourceSense']
            sset_word.append(var[2:-3])

        except:
            sset_word.append('None')

In [12]:
estwnId_list = []
sset_word = []

fetch_estwnId(xml_db)

In [16]:

upload_data(entry_db, xml_db)